In [15]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd


def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "C:\Program Files (x86)\Google\chromedriver"}
    return Browser("chrome", **executable_path)


def scrape_first(url, dicti, tags_class_dict):
    browser = init_browser()
    browser.visit(str(url))
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    for key, value in tags_class_dict.items():
        dicti[key.replace(" ", "_")] = soup.find(value, class_= key).get_text()
        try:
            dicti[key.replace(" ", "_")+"_Style"] = soup.find(value, class_= key)['style']
        except:
            dicti[key.replace(" ", "_")+"_Style"] = "Not Available"
        
    
    browser.quit()
    
    return dicti
    


In [17]:
mars_news = {}
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
news_dict = {"content_title" : "div", "article_teaser_body" : "div"}
       
scrape_first(news_url, mars_news, news_dict)



{'content_title': "Five Things to Know About InSight's Mars Landing",
 'content_title_Style': 'Not Available',
 'article_teaser_body': "NASA engineers will be holding their breath when their spacecraft heads into Mars' atmosphere on Nov. 26.",
 'article_teaser_body_Style': 'Not Available'}

In [3]:
mars_image = {}
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
image_dict = {"carousel_item" : "article"}

scrape_first(featured_image_url, mars_image, image_dict)


{'carousel_item': '\n\n\n\n\n\t\t\t\t  FEATURED IMAGE\n\t\t\t\t\n\n\t\t\t\t  Dwarf Galaxy Spotted\t\t\t\t\n\n\n\n\n\t\t\t\t\tFULL IMAGE\n\t\t\t\t  \n\n\n\n\n',
 'carousel_item_Style': "background-image: url('/spaceimages/images/wallpaper/PIA16613-1920x1200.jpg');"}

In [4]:
jpg_img_path = mars_image["carousel_item_Style"][slice(23,-3,1)]
featured_image_url = "https://www.jpl.nasa.gov"+jpg_img_path

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16613-1920x1200.jpg'

In [5]:
mars_weather = {}
weather_url = "https://twitter.com/marswxreport?lang=en"
weather_dict = {"TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" : "p"}

scrape_first(weather_url, mars_weather, weather_dict)

{'TweetTextSize_TweetTextSize--normal_js-tweet-text_tweet-text': 'Sol 2213 (2018-10-27), high -12C/10F, low -70C/-93F, pressure at 8.74 hPa, daylight 06:11-18:29',
 'TweetTextSize_TweetTextSize--normal_js-tweet-text_tweet-text_Style': 'Not Available'}

In [6]:
mars_weather = mars_weather['TweetTextSize_TweetTextSize--normal_js-tweet-text_tweet-text']
mars_weather

'Sol 2213 (2018-10-27), high -12C/10F, low -70C/-93F, pressure at 8.74 hPa, daylight 06:11-18:29'

In [7]:
facts_url = "https://space-facts.com/mars/"

tabels_facts_url = pd.read_html(facts_url)

facts_df = tabels_facts_url[0]
facts_df.columns = ['info', 'values']
facts_df = facts_df.set_index('info')

In [8]:
facts_df.head(20)

,values
info,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = init_browser()
browser.visit(str(hemi_url))
html = browser.html
soup = BeautifulSoup(html, "lxml")
titles = soup.find_all("h3")
browser.quit()


In [10]:
print(titles)
type(titles)

[<h3>Cerberus Hemisphere Enhanced</h3>, <h3>Schiaparelli Hemisphere Enhanced</h3>, <h3>Syrtis Major Hemisphere Enhanced</h3>, <h3>Valles Marineris Hemisphere Enhanced</h3>]


bs4.element.ResultSet

In [11]:
hemisphere_image_urls = []
for title in titles:
    data = {}
    titl = title.text
    data['title'] = titl
    titl = titl.replace(" ", "_")
    titl = titl.replace("_Hemisphere", "")
    titl = titl.lower()
    data['img_url'] = f"https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/{titl}.tif/full.jpg"
    hemisphere_image_urls.append(data)
    

print(hemisphere_image_urls)




[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [13]:
!jupyter nbconvert --to script scraper.ipynb


[NbConvertApp] Converting notebook scraper.ipynb to script
[NbConvertApp] Writing 2867 bytes to scraper.py
